# FusionDB reactor browser

Lists all reactors side by side (one column per reactor) with every available field, including derived values. Columns with warnings are highlighted in yellow; mismatches beyond the 1% tolerance are highlighted in red (red overrides yellow). Warning messages are echoed below the table.
from fusdb.reactors_class import relations_for  # dynamic relation access


In [1]:
from dataclasses import fields as dataclass_fields
from pathlib import Path
import re
import warnings

import pandas as pd  # Pandas is required for styling

from fusdb.loader import find_reactor_dirs, load_reactor_yaml
from fusdb.reactors_class import Reactor

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

# Load reactors and capture warnings per reactor and field
root = Path('.')
reactors = {}
cell_status: dict[tuple[str, str], str] = {}
computed_flags: dict[tuple[str, str], bool] = {}
warning_messages: dict[str, list[str]] = {}
field_patterns = [
    re.compile(r"Inconsistent values for ([A-Za-z0-9_]+)", re.IGNORECASE),
    re.compile(r"^([A-Za-z0-9_]+) differs", re.IGNORECASE),
    re.compile(r"^([A-Za-z0-9_]+) updated", re.IGNORECASE),
    re.compile(r"(density_profile_file)", re.IGNORECASE),
]

def classify_status(msg: str) -> str:
    text = msg.lower()
    if "within tolerance" in text:
        return "yellow"
    if "inconsistent" in text or "updated from" in text or "not found" in text:
        return "red"
    return "yellow"

for reactor_dir in find_reactor_dirs(root):
    path = reactor_dir / 'reactor.yaml'
    with warnings.catch_warnings(record=True) as caught:
        warnings.simplefilter('always')
        reactor = load_reactor_yaml(path)

    reactors[reactor.id] = reactor
    messages = [str(w.message) for w in caught]
    warning_messages[reactor.id] = messages
    for fname, src_name in getattr(reactor, '_sources', {}).items():
        computed_flags[(fname, reactor.id)] = src_name != 'explicit'

    for msg in messages:
        field = None
        for pat in field_patterns:
            m = pat.search(msg)
            if m:
                field = m.group(1)
                break
        if field is None:
            continue
        cell_status[(field, reactor.id)] = classify_status(msg)

# Build a DataFrame with every dataclass field as rows
field_names = [f.name for f in dataclass_fields(Reactor)]
table_data: dict[str, list[object]] = {}
for rid, reactor in sorted(reactors.items()):
    table_data[rid] = [getattr(reactor, name) for name in field_names]

df = pd.DataFrame(table_data, index=field_names)

def style_cells(data: pd.DataFrame) -> pd.DataFrame:
    styles = pd.DataFrame('', index=data.index, columns=data.columns)
    for field in styles.index:
        for rid in styles.columns:
            parts = []
            if computed_flags.get((field, rid)):
                parts.append('text-decoration: underline;')
            status = cell_status.get((field, rid))
            if status == 'red':
                parts.append('color: #b30000; font-weight: 600;')
            elif status == 'yellow':
                parts.append('color: #b58900;')
            styles.loc[field, rid] = ' '.join(parts)
    return styles

styled = df.style.apply(style_cells, axis=None)
display(styled)

# Show warnings below the table
for rid, msgs in warning_messages.items():
    if not msgs:
        continue
    print(f'Warnings for {rid}:')
    for msg in msgs:
        print(f' - {msg}')


/home/alessmor/Scrivania/fusdb/src/fusdb/reactors_class.py:362: UserWarning: Relation variables ['beta_limit'] are not Reactor fields; they will not be auto-assigned
  warnings.warn(


,ARC_2015,DEMO_2018,STEP2024
id,ARC_2015,DEMO_2018,STEP2024
name,ARC 2015,EU-DEMO 2018,STEP 2024
reactor_configuration,compact tokamak,tokamak,spherical tokamak
organization,MIT,EUROFUSION,UKAEA
reactor_class,ARC-class,DEMO-class,STEP-class
country,USA,EU,UK
site,None,None,None
design_year,2015,2018,2024
doi,10.1016/j.fusengdes.2015.07.008,10.1016/j.fusengdes.2022.113080,10.1088/1741-4326/ad6ea2
notes,Conceptual compact tokamak from MIT.,EU-DEMO Physics Baseline 2018,None


Warnings for ARC_2015:
 - a violates Tokamak volume relation: expected value is 1.1747340088161353, got 1.13 (holding V_p=153.0, R=3.3, kappa=1.84, delta_95=0.0, squareness=0.0)
 - beta_N violates Normalized beta relation: expected value is 2.5323589743589743, got 2.59 (holding beta_T=0.019, a=1.13, B0=9.2, I_p=7.8)
Warnings for STEP2024:
 - kappa violates Elongation 95% relation: expected value is 3.136, got 3.0 (holding kappa_95=2.8)
 - kappa violates ST elongation vs aspect ratio relation: expected value is 2.597675981602159, got 3.0 (holding A=1.8)
 - delta_95 violates ST triangularity vs aspect ratio relation: expected value is 0.39998399634202103, got 0.0 (holding A=1.8)
